# Heat kernel

In [1]:
sigma = 0.1

## Load packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys
path = '../../scripts/'
sys.path.insert(0,path)
from RipserToDict import ripser_to_dict
from PlotPersistence import plot_persistence
import pickle

## Functions

In [3]:
def heat_kernel(F,G,sigma):
    
    """Let F and G be PH matrices of dimensions n_points x 2, where the columns are birth and death respectively.
    The matrices are thus only for one dim each, e.g. 0 or 1 or 2.
    sigma is a scalar parameter which may be taken to be 0.1.
    The output is the heat kernel"""

    kernel = 0
    for z in G:
        for y in F:
            kernel += np.exp(-np.linalg.norm(y - z,ord=2)**2/(8.*sigma))
            kernel += -np.exp(-np.linalg.norm(y - np.flipud(z),ord=2)**2/(8.*sigma))
    kernel *= 1./(8.*np.pi*sigma)

    return kernel

In [4]:
def heat_kernel_distance(F,G,sigma):
    
    """Let F and G be PH matrices of dimensions n_points x 2, where the columns are birth and death respectively.
    The matrices are thus only for one dim each, e.g. 0 or 1 or 2.
    sigma is a scalar parameter which may be taken to be 0.1.
    The output is the heat kernel distance"""
    
    inner = heat_kernel(F,F,sigma) + heat_kernel(G,G,sigma) - 2*heat_kernel(F,G,sigma)
    distance = np.sqrt(heat_kernel(F,F,sigma) + heat_kernel(G,G,sigma) - 2*heat_kernel(F,G,sigma))
    
    return distance

## Create labels and train kernel

In [5]:
# load train set
train_seeds = range(0,5)
train_persistences = []
train_labels = []
for seed in train_seeds:
    
    # sphere
    with open('../../../heavy_files/exercises/ml_on_1_to_3/sphere_persistences/' + str(seed) + '.txt','rb') as f:
        train_persistences.append(pickle.load(f))
    train_labels.append(0)
    
    # torus
    with open('../../../heavy_files/exercises/ml_on_1_to_3/torus_persistences/' + str(seed) + '.txt','rb') as f:
        train_persistences.append(pickle.load(f))
    train_labels.append(1)

In [6]:
train_kernel = np.zeros((len(train_labels),len(train_labels)))
train_homology = [train_persistences[x][1] for x in train_seeds]
for x, i in enumerate(train_homology):
    for y, j in enumerate(train_homology):
        train_kernel[x,y] = heat_kernel(i,j,sigma)

In [7]:
# export
with open('train_kernel.txt','wb') as f:
    pickle.dump(train_kernel,f)

with open('train_labels.txt','wb') as f:
    pickle.dump(train_labels,f)

## Create test kernel and test labels

In [8]:
# load test set
test_seeds = range(5,10)
test_persistences = []
test_labels = []
for seed in test_seeds:
    
    # sphere
    with open('../../../heavy_files/exercises/ml_on_1_to_3/sphere_persistences/' + str(seed) + '.txt','rb') as f:
        test_persistences.append(pickle.load(f))
    test_labels.append(0)
    
    # torus
    with open('../../../heavy_files/exercises/ml_on_1_to_3/torus_persistences/' + str(seed) + '.txt','rb') as f:
        test_persistences.append(pickle.load(f))
    test_labels.append(1)

In [9]:
test_kernel = np.zeros((len(test_labels),len(test_labels)))
test_homology = [test_persistences[x][1] for x in range(len(test_seeds))]
for x, i in enumerate(test_homology):
    for y, j in enumerate(test_homology):
        test_kernel[x,y] = heat_kernel(i,j,sigma)

In [10]:
# export
with open('test_kernel.txt','wb') as f:
    pickle.dump(test_kernel,f)

with open('test_labels.txt','wb') as f:
    pickle.dump(test_labels,f)